In [55]:
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder, HuggingFaceEncoder
from semantic_router import RouteLayer
import pandas as pd

In [56]:
# Load the dataframes json files
df_synthetic = pd.read_json("synthetic_intetions.json")

X_syn = df_synthetic[['Id','Message']]
y_syn = df_synthetic['Intention'].to_list()

In [64]:
from sklearn.model_selection import train_test_split

# Split the dataset with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_syn, y_syn, test_size=0.1, random_state=0, stratify=y_syn
)

In [65]:
# Replace "None" with None
y_train = [None if i == "None" else i for i in y_train]
y_test = [None if i == "None" else i for i in y_test]

In [66]:
user_intentions = ["manage_personal_info", "search_scholarships_and_internationals","search_universities" , "matchmaking", "query_matches", "leverage_rag", "company_info", None]

In [70]:
MESSAGES_INFO = {
}

for intention in user_intentions:
    MESSAGES_INFO[intention] = []

for message, label in zip(X_test["Message"], y_test):
    for intention in user_intentions:
        if label == intention:
            MESSAGES_INFO[intention].append(message)
            continue

MESSAGES_INFO

{'manage_personal_info': ['Set my country to the United Kingdom.',
  'I need to set my country to Germany.',
  'Change my education level to doctoral.',
  "Change my education level to master's.",
  'Add a preference for scholarships.'],
 'search_scholarships_and_internationals': ['Can you list some scholarships for tech students?',
  'What are the eligibility criteria for scholarships?',
  'What international scholarships can I apply for?',
  'Can you give me tips on applying for scholarships?',
  'What scholarships are available for online degree programs?',
  'How do I find scholarships for studying in Europe?'],
 'search_universities': ['Can you tell me about universities in Australia?',
  'What universities offer computer science programs?',
  'Can you list universities with strong internship programs?',
  'What is the application process for MIT?',
  'Can you find universities that specialize in arts?'],
 'matchmaking': ['Please match my university preferences with available opti

In [71]:
manage_personal_info = Route(
    name="manage_personal_info",
    description="The user wants to manage his personal information, such as username, age, password, et cetera...",
    utterances=MESSAGES_INFO['manage_personal_info'],
)

company_info = Route(
    name="company_info",
    description="The user wants to know information about the company called 'UniMatch'.",
    utterances=MESSAGES_INFO['company_info'],
)

leverage_rag = Route(
    name="leverage_rag",
    description="The user intends wants the chatbot to extract information from an external source (PDF or website).",
    utterances=MESSAGES_INFO['leverage_rag'],
)

matchmaking = Route(
    name="matchmaking",
    description="The user wants to make matches for his university, specifying some preferences.",
    utterances=MESSAGES_INFO['matchmaking'],
)

query_matches = Route(
    name="query_matches",
    description="The user wishes to access and see the previously-made matches.",
    utterances=MESSAGES_INFO['query_matches'],
)

search_scholarships_and_internationals = Route(
    name="search_scholarships_and_internationals",
    description="The user wishes to search for scholarships or international opportunities.",
    utterances=MESSAGES_INFO['search_scholarships_and_internationals'],
)

search_universities = Route(
    name="search_universities",
    description="The user wishes to search for universities or courses.",
    utterances=MESSAGES_INFO['search_universities'],
)
routes = [manage_personal_info, company_info, leverage_rag, matchmaking, query_matches, search_scholarships_and_internationals, search_universities]



# Baseline Routers

In [76]:
# Standard HF-Encoded RL
encoderHF = HuggingFaceEncoder()
hf_rl = RouteLayer(encoder=encoderHF, routes=routes) #aggregation = "mean", "max" or "sum". #top_k = 5

accuracy = hf_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

# Call the fit method on HFEncoder
hf_rl.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)
accuracy = hf_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy (Post-fit): {accuracy*100:.2f}%")


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 85.37%


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy (Post-fit): 85.37%


In [77]:
# Standard OA-Encoded RL
encoderOA = OpenAIEncoder()
oa_rl = RouteLayer(encoder=encoderOA, routes=routes) #aggregation = "mean", "max" or "sum" #top_k = 5

accuracy = oa_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

# OpenAIEncoder Fit
oa_rl.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)
accuracy = oa_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy (Post-fit): {accuracy*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 92.68%


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy (Post-fit): 92.68%


In [23]:
# Result: Choose OA encoder as it has best score

# Tuning Selected Router

## Aggregate

In [78]:
oa_rl_mean = RouteLayer(encoder=encoderOA, routes=routes, aggregation='mean') #aggregation = "mean", "max" or "sum" #top_k = 5
oa_rl_max = RouteLayer(encoder=encoderOA, routes=routes, aggregation='max') #aggregation = "mean", "max" or "sum" #top_k = 5
oa_rl_sum = RouteLayer(encoder=encoderOA, routes=routes, aggregation='sum') #aggregation = "mean", "max" or "sum" #top_k = 5


In [82]:
oa_rl_mean.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [83]:
oa_rl_max.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [84]:
oa_rl_sum.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [85]:
accuracy_mean = oa_rl_mean.evaluate(X=X_test["Message"].to_list(), y=y_test)
accuracy_max = oa_rl_max.evaluate(X=X_test["Message"].to_list(), y=y_test)
accuracy_sum = oa_rl_sum.evaluate(X=X_test["Message"].to_list(), y=y_test)

print(f"Accuracy mean: {accuracy_mean*100:.2f}%")
print(f"Accuracy max: {accuracy_max*100:.2f}%")
print(f"Accuracy sum: {accuracy_sum*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy mean: 97.56%
Accuracy max: 97.56%
Accuracy sum: 92.68%


In [86]:
oa_rl_max_2 = RouteLayer(encoder=encoderOA, routes=routes, aggregation='max', top_k=10) #aggregation = "mean", "max" or "sum" #top_k = 5
oa_rl_max_5 = RouteLayer(encoder=encoderOA, routes=routes, aggregation='max', top_k = 20) #aggregation = "mean", "max" or "sum" #top_k = 5
oa_rl_max_10 = RouteLayer(encoder=encoderOA, routes=routes, aggregation='max', top_k = 30) #aggregation = "mean", "max" or "sum" #top_k = 5



In [87]:
oa_rl_max_2.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [89]:
oa_rl_max_5.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [90]:
oa_rl_max_10.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [91]:
accuracy_2 = oa_rl_max_2.evaluate(X=X_test["Message"].to_list(), y=y_test)
accuracy_5 = oa_rl_max_5.evaluate(X=X_test["Message"].to_list(), y=y_test)
accuracy_10 = oa_rl_max_10.evaluate(X=X_test["Message"].to_list(), y=y_test)

print(f"Accuracy k=2: {accuracy_2*100:.2f}%")
print(f"Accuracy k=5: {accuracy_5*100:.2f}%")
print(f"Accuracy k=10: {accuracy_10*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy k=2: 97.56%
Accuracy k=5: 97.56%
Accuracy k=10: 97.56%


# Final RL


In [93]:
final_rl = RouteLayer(encoder=encoderOA, routes=routes, aggregation='max')
final_rl.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

# Print metrics

In [26]:
def evaluate_router(rl):
    REPORT_DATA = {
}
    for intention in user_intentions:
        REPORT_DATA[intention] = [0, 0]

    for (index, row), label in zip(X_test.iterrows(), y_test):
        message = row["Message"]
        prediction = rl(message)

        total, wrong = REPORT_DATA[label]
        total += 1
        
        if prediction.name == label:
            pass
        else:
            wrong += 1
        REPORT_DATA[label] = [total, wrong]
    
    return REPORT_DATA


In [94]:
pd.DataFrame(evaluate_router(final_rl)).T.to_excel('evaluation_results.xlsx')

ValueError: No engine for filetype: 'csv'

In [146]:
REPORT_DATA = {
}

In [147]:
for intention in user_intentions:
    REPORT_DATA[intention] = [0, 0]


In [148]:
for (index, row), label in zip(X_test.iterrows(), y_test):
    message = row["Message"]
    prediction = final_rl(message)

    total, wrong = REPORT_DATA[label]
    total += 1
    
    if prediction.name == label:
        pass
    else:
        wrong += 1
    REPORT_DATA[label] = [total, wrong]


In [154]:
final_rl('I want to change my password')

RouteChoice(name=None, function_call=None, similarity_score=None)

In [102]:
for (index, row), label in zip(X_train.iterrows(), y_train):
    message = row["Message"]
    prediction = final_rl(message)

    if label == 'None':
        label = None
    
    if prediction.name == label:
        continue
    else:
        print(prediction, label)


name=None function_call=None similarity_score=None manage_personal_info
name=None function_call=None similarity_score=None manage_personal_info


KeyboardInterrupt: 

In [149]:
pd.DataFrame(REPORT_DATA).T

,0,1
manage_personal_info,5,4
search_scholarships_and_internationals,6,0
search_universities,5,5
matchmaking,5,0
query_matches,6,0
leverage_rag,5,5
company_info,6,6
None,3,0


# Deploy

In [92]:
final_rl.to_json("layer.json") # This one avoids overfitting the most

2024-12-22 18:56:17 INFO semantic_router.utils.logger Saving route config to layer.json
